# Document Scanner

In [26]:
import cv2
import numpy as np
height = 640
width = 480
cap = cv2.VideoCapture(1)
cap.set(0, height) #Width
cap.set(4, width) #Height
cap.set(10, 100) #Brightness
def preProcessing(img):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (1, 1), 1)
    imgCanny = cv2.Canny(imgBlur, 100, 220)
    cv2.imshow("Test", imgCanny)
    kernel = np.ones((2, 2))
    imgDilate = cv2.dilate(imgCanny, kernel, iterations = 2)
    imgErode = cv2.erode(imgDilate, kernel, iterations = 1)
    imgThreshold = imgErode
    cv2.imshow("Thresh", imgThreshold)
    return imgThreshold
def getContours(img):
    biggest = np.array([])
    maxArea = 0
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 5000:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 0), 3)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            if(area > maxArea and len(approx) == 4):
                maxArea = area
                biggest = approx
    cv2.drawContours(imgContour, biggest, -1, (255, 0, 0), 10)
    #print(biggest)
    return biggest
def reorder(points):
    points = points.reshape((4, 2))
    pointsReordered = np.zeros((4,1,2), np.int32)
    
    add = points.sum(1)
    pointsReordered[0] = points[np.argmin(add)]
    pointsReordered[3] = points[np.argmax(add)]
    
    diff = np.diff(points, axis = 1)
    pointsReordered[1] = points[np.argmin(diff)]
    pointsReordered[2] = points[np.argmax(diff)]
    return pointsReordered
def getWarp(img, biggest):
    biggest = reorder(biggest)
    pt1 = np.float32(biggest)
    pt2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    matrix = cv2.getPerspectiveTransform(pt1, pt2)
    imgOutput = cv2.warpPerspective(img, matrix, (width, height))
    
    imgCropped = imgOutput[20:imgOutput.shape[0]-20, 20:imgOutput.shape[1]-20]
    imgCropped = cv2.resize(imgCropped, (width, height))
    return imgCropped
    #pass
while True:
    success, img = cap.read()
    imgContour = img.copy();
    imgThreshold = preProcessing(img)
    biggest = getContours(imgThreshold)
#     print(biggest.shape)
#     print(biggest)
    if biggest.size != 0:
        imgWarped = getWarp(img, biggest)
        cv2.imshow("Warped", imgWarped)
    #cv2.imshow("Video", img)q
    cv2.imshow("Contour", imgContour)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()